# Intro

# Multivariate Regression 

Also known as 'generalised linear model', it generalises linear regression to multiple input variables (regressors) and $n$ observations. It  is best expressed in matrix form as:

$$
Y = X \beta + \epsilon
$$

where $Y$ is a vector representing the endogenous (dependent) variable, $X$ is a matrix representing the exogenous (independent) variables, $\beta$ is the coefficients vector and $\epsilon$ the residuals vector. 

The OLS method, which minimizes the sum of squared residuals via the Maximum Likelihood Estimation method (MLE), is used to estimate the parameters:

$$
\begin{align}
\hat{\beta} = (X^\prime X)^{-1} X^\prime Y \\
\hat{\epsilon} = Y - X \hat{\beta}
\end{align}
$$

The covariance matrix of the residuals estimate is:

$$
\hat{\Sigma} = scale \times  \sum \epsilon \epsilon^\prime
$$
where $scale = 1/ n$ using the MLE estimator or $scale= 1/ (n - kp)$ using the OLS estimator for a model with $k$ variables and $p$ lags.

The covariance matrix for the coefficients is:

$$
(X X^\prime)^{-1} \otimes \hat{\Sigma}
$$
where $\otimes$ is the _Kronecker product_. 

The Log Likelihood Function for OLS is:

$$
\log(L) = -\frac{n}{2}\log (2 \pi) -\frac{n}{2}\log | \hat{\Sigma}| - \frac{n}{2}
$$

The variance of the residuals and parameters are therefore the diagonal elements of the corresponding covariance matrices, from which the standard errors can be calculated. 

These are the conventions used in the script _analysis.py_. Additional mathematical details can be found in [wiki VAR](https://en.wikipedia.org/wiki/Vector_autoregression).

The are several assumptions about the nature of the variables  in the model to hold. In particular, these should be _stationary_ and the $\epsilon$ homoscedastic (with finite variance) and normally distributed.

The main applications of the multivariate regression are:
* Vector Autoregression Models - also known as VAR(p), these can be used to forecast, test for stationarity (e.g. ADF test) or model stationary series like returns
* Error Correction Model - also known as ECM, these are used to model series which aren't stationary or that have stochastic trends, like prices

## Vector Autoregression - VAR(p)

Also referred to as VAR(p) where _p_ is the lag order, the is simply a linear regression on a time series and its lagged (past) values:

$$
\begin{align}
Y_t = c + \sum^{p}_{i=1} \phi_i Y_{t-i} + \epsilon_t \\
\end{align}
$$

where c is a constant (also known as the drift term), $\phi_i$ are the parameters of the model and $\epsilon_t$ is the error term. Whether to exclude the constant $c$ or not depends on the nature of what we are trying to model.

Computationally, the model can be fitted in one go by using the OLS method described above with a special matrix formulation. Example code for this can be found in _analysis.py_ in the project repository.

As shown in Section X, the VAR(p) system can be rewritten in terms of differences and lagged differeces, which is how it is commonly expressed in some cases, for example in the ADF test. See more details in [wiki VAR](https://en.wikipedia.org/wiki/Vector_autoregression), [wiki AR](https://en.wikipedia.org/wiki/Autoregressive_model)


# Dickey-Fuller Test and ADF

The _Dickey-Fuller test_ tests the null hypothesis of whether a unit root is present in the autoregressive model (also known as AR(p), see Appendix) of the time series. For example, a simple AR(1) model is:

$$
Y_t = \beta Y_{t-1} + \epsilon_t
$$

If $\beta=1$ the series is said to have a 'unit root' and is hence non-stationary. The equation can be rewriten as:

$$
\Delta Y_t = (\beta - 1) Y_{t-1} + \epsilon_t = \phi Y_{t-1} + \epsilon_t
$$

where $\phi= \beta-1$. Hence, testing for unit root is equivalent to testing $\phi=0$. 

The value of the test statistic $\hat{\phi}/std.err(\hat{\phi})$ is then compared to the relevant critical values for the Dickey-Fuller distribution. If found lower, then the null hypothesis $\phi=0$ is rejected and the series can be considered stationary.

There are 3 main versions of the test depending on whether drift and/or time-dependent terms are included:

* Test for a unit root: $\Delta Y_t = \phi Y_{t-1} + \epsilon_t$
* Test for a unit root with drift: $\Delta Y_t = c_0 + \phi Y_{t-1} + \epsilon_t$
* Test for a unit root with drift and deterministic time trend: $\Delta Y_t = c_0 + c_1 t + \phi Y_{t-1} + \epsilon_t$

Each version of the test has its own critical values which depends on the size of the sample. Which version to use isn't straightforward and the wrong choice can lead to wrong conclusions. In general, financial time series exclude the time trend.

There is an extension to the test called the **Augmented Dickey–Fuller test (ADF)**, which removes autocorrelation effects by including lagged difference terms $\phi_p \Delta Y_{t-p}$.  The optimal lag oder could then be determined from the information criteria (see above). 

Clearly the above equations belong to the family of generalised linear models, which means the parameters can be estimated using the familiar linear regression  described above.

Additional details on this topic can be found in [wiki ADF](https://en.wikipedia.org/wiki/Augmented_Dickey%E2%80%93Fuller_test), [wiki DF](https://en.wikipedia.org/wiki/Dickey%E2%80%93Fuller_test) and the Appendix.

### Optimal Lag Order

To select the optimal lag order, one approach uses the Akaike Information Criterion (AIC). Iterating over different lag orders, the one yielding the lowest value of AIC is selected. Statsmodels suggest to try up to a maximum lag order of $12*(n/100)^{1/4}$ where $n$ is the number of observations.

There are different definitions of AIC used - we use the same as in  [statstools](http://statsmodels.sourceforge.net/stable/vector_ar.html#lag-order-selection), which has different definitions  for AR(p) and ADF test as:

$$
\begin{equation}
AIC = \log | \hat{\Sigma} |+ 2 \frac{1 + k}{n} \qquad \mathrm{(AR(p) \ model)} \\
AIC = -2 \log(L) + 2k  \qquad \mathrm{(ADF \ test)} 
\end{equation}
$$

where $k$ is the number of estimated parameters. Other information criteria can be used, see for example [ref](https://en.wikipedia.org/wiki/Information_criterion).


### Stability Condition

It requires for the eigenvalues of the estimated coefficients matrix or vector to be
inside the unit circle ($ < 1$):
$$
| \lambda I - \hat{\beta} | = 0
$$
This is equivalent to requiring the roots of the characteristic polynomial of the AR(p) system to be outside the unit circle - see [ref](http://matthieustigler.github.io/Lectures/Lect2ARMA.pdf).

# Trading Strategies

* trading around the spread $e_t$ where estimated weights $\beta_i$ represent the position to take on each stock
* optimised bounds give entre/exit signals, e.g. $\mu_e \pm 1\sigma_e$. $\sigma_e$ can be obtained from fitting to OU process or using optimisation 
* speed of mean-reversion in the spread $\theta$ gives idea of profitability over time and can be converted to half-life (expected position holding time) as $\tilde{\tau} \propto ln 2 / \theta$
* the dollar MtM P&L $\Delta e_t = e_t - e_{t-1} $ is independent of its mean $\mu_e$
* Trading half-life strategy: requires entering at a bound an exiting on the mean, generating half the possible 'aplha'. This would reflect also the time it would take to generate P&L, so a high $\theta$ is desirable
* For trade design, long waiting times mean cost of carry (financing of the leverage) and increase the risk of regime change in the co-integrating relationship. Estimate half-life time of a co-integrating relationship returning to its equilibrium and explore trading partial half life.

---> EXTRA: Cointegration is a filter on data - mean-reversion is of lower frequency than the data

## Naive Beta Hedging

$$
\begin{array}{rl}
\mathrm{E} [ \Delta Y_{gasoil}^{is}  -0.6282 \Delta X_{brent}^{is}] =  & \alpha   =  -0.0915 \qquad (R^2 = 0.354) \\
\mathrm{E} [ \Delta Y_{gasoil}^{is}  - \Delta X_{brent}^{is}] =  & \alpha   =   -0.0185  \qquad  \text{(no fit)} \\
\mathrm{E} [ \Delta Y_{gasoil}^{is}]  =  & \alpha   =   
-0.2148  \qquad  \text{(no fit)}
\end{array}
$$


As 'delta hedging' is for options, 'beta hedging' involves reducing the overall exposure ('beta') of a portfolio by purchasing stocks with offsetting betas.

Using returns which are more stationary than price levels, we experiment by running a simple linear regression on variables we think are correlated and use the estimated beta coefficient to take positions. In the case off Gasoil, asssume the returns were dependent on the market via:

$$\Delta Y_{gasoil} = \beta \Delta X_{brent} +  \alpha + \epsilon_t $$

We could take a short position in Brent equal to $\beta \Delta X_{brent}$ to try to eliminate the _risk_ in our Gasoil position. We would expect our returns to be on average:

$$\mathrm{E} [\Delta Y_{gasoil} - \beta X_{brent}] = \alpha $$

i.e. purely defined by 'alpha'.  For the in-sample data, running the above regression would actually yield negative returns, although less severe than just going long Gasoil:

$$
\begin{array}{rl}
\mathrm{E} [ \Delta Y_{gasoil}^{is}  -0.6282 \Delta X_{brent}^{is}] =  & \alpha   =  -0.0915 \qquad (R^2 = 0.354) \\
\mathrm{E} [ \Delta Y_{gasoil}^{is}  - \Delta X_{brent}^{is}] =  & \alpha   =   -0.0185  \qquad  \text{(no fit)} \\
\mathrm{E} [ \Delta Y_{gasoil}^{is}]  =  & \alpha   =   
-0.2148  \qquad  \text{(no fit)}
\end{array}
$$

This highlights the problem that the estimated beta is not constant as we walk forward in time. As such, the short position we took out in Brent is not perfectly hedging our portfolio. 

Surprisingly, our performance in the out-of-sample seems to be better using the in-sample fit but that seems to be driven by 'lucky' positive fluctuations in the data:

$$
\begin{array}{rl}
\mathrm{E} [ \Delta Y_{gasoil}^{os}  +0.0915  -0.6282 \Delta X_{brent}^{os}] =  & \mathrm{E}[ \alpha_t]    = 0.11378   \\
\mathrm{E} [ \Delta Y_{gasoil}^{os}  - \Delta X_{brent}^{os}] =   & \mathrm{E}[ \alpha_t]   =  0.00115  \qquad  \text{(no fit)}
\end{array}
$$

## Pairs trading

* Go "Long" the spread when it goes below $\mu_e - \sigma_{eq}$: this means longing $Y$ and shorting $X$

* Go "Short" the spread when it goes above $\mu_e + \sigma_{eq}$: this means shorting $Y$ and longing $X$

* Exit positions when the spread crosses the equilbrium-level $\mu_e$
